<a href="https://colab.research.google.com/github/MasterSlyer10/CSMODEL/blob/main/MCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Dataset Description



In [ ]:
stroke_df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [ ]:
stroke_df.info()

# Data Cleaning

In [ ]:
stroke_df.isnull().any()

In [ ]:
stroke_df['bmi'].isnull().sum()

In [ ]:
stroke_df.shape

In [ ]:
stroke_df = stroke_df.dropna(subset='bmi')

In [ ]:
stroke_df.shape

# Exploratory Data Analysis

# Research Question